In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import random

In [20]:
print("Load labels")
label_file = pd.read_csv("./train/labels_ext.csv", index_col=0)
Xtrain = pd.read_csv("./train/joint_accident_person_train_ext.csv", index_col=0)
Xtest = pd.read_csv("./test/joint_accident_person_test_ext.csv", index_col=0)

Load labels


In [21]:
ID= Xtest['ID'].astype(np.int64)

temp = pd.merge(Xtrain, label_file, on=['ID'], how='inner')
y = temp['DRUNK_DR']

In [23]:
Xtrain.drop('ID', axis= 1, inplace=True)
train_x = Xtrain
train_y = y
test_x = Xtest.drop('ID', axis= 1, inplace=True)

In [25]:
Xtrain

,VE_FORMS,PEDS,PERSONS,NHS,LATITUDE,LONGITUD,HARM_EV,CF1,CF2,CF3,...,INJ_SEV_1,INJ_SEV_2,INJ_SEV_3,INJ_SEV_4,INJ_SEV_5,INJ_SEV_6,INJ_SEV_8,INJ_SEV_9,SPEC_USE_13.0,IMPACT1_19.0
0,1,0,1,0,32.992778,-86.714469,34,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0,1,0,34.720764,-85.937033,34,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0,1,0,32.561533,-85.963558,37,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,0,1,0,33.184503,-85.360019,42,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2,0,3,1,31.638442,-86.787378,12,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,2,0,3,1,31.638442,-86.787378,12,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,2,0,3,1,31.638442,-86.787378,12,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,1,0,1,0,32.249086,-86.093311,34,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,2,0,3,0,34.224758,-86.829319,12,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,2,0,3,0,34.224758,-86.829319,12,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
trainX = train_x.as_matrix()
trainY = train_y.astype(int)
testX = test_x.as_matrix()

AttributeError: 'NoneType' object has no attribute 'as_matrix'

In [6]:
dtrain = xgb.DMatrix(trainX, missing=-1, label=trainY)
dtest = xgb.DMatrix(testX, missing=-1)

### Parameter tuning
1. for `scale_pos_weight`, read more [here](https://github.com/dmlc/xgboost/blob/master/demo/kaggle-higgs/speedtest.py)
2. If you care about predicting the right probability, which in such a case, you cannot re-balance the dataset. Set parameter `max_delta_step` to a finite number (say 1) will help convergence. Read more [here](https://github.com/dmlc/xgboost/blob/master/doc/param_tuning.md)

In [7]:
# parameter tuning
# random.seed(1024)

param = {}
param['objective'] = 'binary:logistic'

# scale weight of positive examples
# rescale weight to make it same as test set
# testsize = 543015
# weight = dtrain[:,31] * float(test_size) / len(train_y)
# param['scale_pos_weight'] = sum_wneg/sum_wpos

ratio = float(np.sum(train_y == 1)) / np.sum(train_y == 0)
param['scale_pos_weight'] = ratio

param['eta'] = 0.05
param['max_depth'] = 7
param['eval_metric'] = 'auc'
# param['silent'] = 1
param['min_child_weight'] = 1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['nthread'] = 8

# Sean:: can't understand how to get dtrain[:, 31]=weight !!
# def fpreproc(dtrain, dtest, param):
#     label = dtrain.get_label()
#     ratio = float(np.sum(label == 0)) / np.sum(label==1)
#     print("ratio= ", ratio)
#     param['scale_pos_weight'] = ratio
#     wtrain = dtrain.get_weight()
#     wtest = dtest.get_weight()
#     sum_weight = sum(wtrain) + sum(wtest)
#     wtrain *= sum_weight / sum(wtrain)
#     wtest *= sum_weight / sum(wtest)
#     print("final wtrain, wtest = ", wtrain, wtest)
#     dtrain.set_weight(wtrain)
#     dtest.set_weight(wtest)
#     return (dtrain, dtest, param)

# XGBoostError: b'[13:07:13] src/metric/rank_metric.cc:36: 
# Check failed: (info.weights.size()) == (ndata) we need weight to evaluate ams'
#
# bst_cv = xgb.cv(param, dtrain, num_boost_round=50, nfold=5, metrics={'ams@0.15', 'auc'}, seed=0)
# bst_cv

In [ ]:
bst_cv = xgb.cv(param, dtrain, num_boost_round=1000, nfold=5, seed=0)

In [12]:
tst = pd.DataFrame(bst_cv)
test = tst['train-auc-mean']-tst['train-auc-std']
test.idxmax()

999

In [13]:
num_round = test.idxmax()

In [14]:
bst = xgb.train(param, dtrain, num_round)
bst

In [15]:
ypred = bst.predict(dtest)

In [18]:
ypred

array([ 0.48696449,  0.20320323,  0.40755364, ...,  0.01393417,
        0.01363044,  0.01220728], dtype=float32)

### Averaging the votes in prediction

In [36]:
predict_df = pd.DataFrame(data={'ID': alltest_df['ID'], 'DRUNK_DR': ypred})
grouped_predict = predict_df.groupby('ID', as_index=False).mean()

### Given a prediction, create a Kaggle submission file


In [37]:
grouped_predict.to_csv('fars_submit_xgb004_production_weighted.csv', index = False)